In [1]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

class PDPTWInstance:
    def __init__(self, n, map_size, speed, extra_time,seed=None):
        """
        初始化 PDPTW 实例
        :param n: pickup 点的数量
        :param map_size: 地图大小
        :param speed: 车辆速度
        :param extra_time: delivery 点时间窗口起始时间的额外时间
        :param seed: 随机数种子
        """
        self.n = n
        self.map_size = map_size
        self.speed = speed
        self.extra_time = extra_time
        # coordinates
        self.depot = (0, 0)  # depot 位于原点
        self.pickup_points = []  # pickup 点的坐标
        self.delivery_points = []  # delivery 点的坐标
        # time
        self.time_windows = []  # 时间窗口列表
        self.service_times = []  # 服务时间列表
        # demand
        self.demands = []  # 需求量列表

        # 设置随机数种子
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        # 生成所有点的索引列表
        self.indices = [0] + list(range(1, self.n+1)) + list(range(self.n+1, 2*self.n+1))
        
        self.generate_points()  # 生成 pickup 和 delivery 点
        self.generate_time_windows()  # 生成时间窗口和服务时间
        self.generate_demands()  # 生成需求量

    def generate_points(self):
        """
        生成 pickup 和 delivery 点
        """
        for _ in range(self.n):
            # 在地图范围内随机生成 pickup 点坐标
            pickup_x = random.uniform(-self.map_size, self.map_size)
            pickup_y = random.uniform(-self.map_size, self.map_size)
            self.pickup_points.append((pickup_x, pickup_y))

            # 在地图范围内随机生成 delivery 点坐标
            delivery_x = random.uniform(-self.map_size, self.map_size)
            delivery_y = random.uniform(-self.map_size, self.map_size)
            self.delivery_points.append((delivery_x, delivery_y))

    def plot_instance(self):
        """
        绘制 PDPTW 实例图
        """
        plt.figure(figsize=(8, 8))
        # 绘制 depot
        plt.scatter(self.depot[0], self.depot[1], c='red', marker='s', s=100, label='Depot')
        # 绘制 pickup 点
        plt.scatter([p[0] for p in self.pickup_points], [p[1] for p in self.pickup_points], c='blue', marker='o', label='Pickup')
        # 绘制 delivery 点
        plt.scatter([d[0] for d in self.delivery_points], [d[1] for d in self.delivery_points], c='green', marker='d', label='Delivery')
        
        # 为每个点添加标签
        for i in range(self.n):
            plt.annotate(f'P{i+1}', (self.pickup_points[i][0], self.pickup_points[i][1]), textcoords='offset points', xytext=(0,5), ha='center')
            plt.annotate(f'D{i+1}', (self.delivery_points[i][0], self.delivery_points[i][1]), textcoords='offset points', xytext=(0,5), ha='center')

        # 设置坐标轴范围
        plt.xlim(-self.map_size-1, self.map_size+1)
        plt.ylim(-self.map_size-1, self.map_size+1)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('PDPTW Instance')
        plt.legend()
        plt.grid(True)
        plt.show()
        
    def calculate_distance_matrix(self):
        """
        计算距离矩阵
        :return: 距离矩阵
        """
        points = [self.depot] + self.pickup_points + self.delivery_points
        num_points = len(points)
        distance_matrix = np.zeros((num_points, num_points))
        
        # 计算每对点之间的欧几里得距离
        for i in range(num_points):
            for j in range(num_points):
                distance_matrix[i][j] = np.sqrt((points[i][0] - points[j][0])**2 + (points[i][1] - points[j][1])**2)
        
        return distance_matrix
    
    def calculate_time_matrix(self):
        """
        计算时间矩阵
        :return: 时间矩阵
        """
        distance_matrix = self.calculate_distance_matrix()
        time_matrix = (distance_matrix / self.speed)*60
        return time_matrix
    
    def generate_time_windows(self):
        """
        生成时间窗口和服务时间
        """
        time_matrix = self.calculate_time_matrix()
        
        # Depot 的时间窗口设置为 [0, inf]
        self.time_windows.append((0, float('inf')))
        
        for i in range(self.n):
            # Pickup 点的起始时间在 [0, 120] 范围内随机生成，结束时间设置为 inf
            pickup_start_time = random.randint(0, 120)
            self.time_windows.append( (pickup_start_time, float('inf')) )
            
            # 每个 pickup 点和 delivery 点的服务时间在 [5, 10] 范围内随机生成一个整数
            self.service_times.append(random.randint(5, 10))
        
        for i in range(1,self.n + 1):
            pickup_start_time = self.time_windows[i][0]
            # Delivery 点的起始时间为对应 pickup 点的起始时间 + 时间矩阵中的时间 + 额外时间，向上取整
            # 结束时间为起始时间 + 30 分钟，向下取整
            delivery_start_time = math.ceil(pickup_start_time + time_matrix[i][i+self.n] + self.extra_time)
            delivery_end_time = math.floor(delivery_start_time + 60)
            self.time_windows.append((delivery_start_time, delivery_end_time))
            
            # 每个 pickup 点和 delivery 点的服务时间在 [5, 10] 范围内随机生成一个整数
            self.service_times.append(random.randint(5, 10))
        
        # Depot 的服务时间设置为 0
        self.service_times.append(0)
        
    def generate_demands(self):
        """
        生成需求量
        """
        # Depot 的需求量为 0
        self.demands.append(0)
        
        for _ in range(self.n):
            # Pickup 点的需求量为 1
            self.demands.append(1)
        for _ in range(self.n):
            # Delivery 点的需求量为 -1
            self.demands.append(-1)
            
    def to_dataframe(self):
        """
        将 PDPTW 实例转换为 pandas 数据框
        :return: pandas 数据框
        """
        data = []
        
        for i in range(len(self.indices)):
            if i == 0:
                point_type = 'd'
                x, y = self.depot
                partner_id = 0
            elif i <= self.n:
                point_type = 'cp'
                x, y = self.pickup_points[i-1]
                partner_id = i + self.n
            else:
                point_type = 'cd'
                x, y = self.delivery_points[i-self.n-1]
                partner_id = i - self.n
                
            data.append([
                i,
                point_type,
                x,
                y,
                self.demands[i],
                self.time_windows[i][0],
                self.time_windows[i][1],
                self.service_times[i],
                partner_id
            ])
        
        columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
        df = pd.DataFrame(data, columns=columns)
        
        return df

In [3]:
import numpy as np
import matplotlib.pyplot as plt

class PDPTWSolution:
    def __init__(self, instance, vehicle_capacity, battery_capacity, battery_consume_rate, routes):
        """
        初始化 PDPTWSolution 对象
        :param instance: PDPTWInstance 对象
        :param vehicle_capacity: 车辆容量
        :param battery_capacity: 电池容量
        :param battery_consume_rate: 电池消耗率
        :param routes: 给定的路径列表
        """
        self.instance = instance
        self.vehicle_capacity = vehicle_capacity
        self.battery_capacity = battery_capacity
        self.battery_consume_rate = battery_consume_rate
        self.routes = routes
        self.num_vehicles = len(routes)
        # battery and capacity
        self.route_battery_levels = np.empty((self.num_vehicles,), dtype=object)
        self.route_capacity_levels = np.empty((self.num_vehicles,), dtype=object)
        # time
        self.route_arrival_times = np.empty((self.num_vehicles,), dtype=object)
        self.route_leave_times = np.empty((self.num_vehicles,), dtype=object)
        self.route_wait_times = np.empty((self.num_vehicles,), dtype=object)
        # evaluation
        self.total_travel_times = np.zeros((self.num_vehicles,))
        self.total_delay_times = np.zeros((self.num_vehicles,))
        self.total_wait_times = np.zeros((self.num_vehicles,))

        self.initialize_routes()
        self.calculate_all()

    def initialize_routes(self):
        """
        初始化与路径相关的变量
        """
        for vehicle_id in range(self.num_vehicles):
            route = self.routes[vehicle_id]
            self.route_battery_levels[vehicle_id] = np.zeros((len(route),))
            self.route_capacity_levels[vehicle_id] = np.zeros((len(route),))
            self.route_arrival_times[vehicle_id] = np.zeros((len(route),))
            self.route_leave_times[vehicle_id] = np.zeros((len(route),))
            self.route_wait_times[vehicle_id] = np.zeros((len(route),))

    def calculate_all(self):
        """
        计算所有状态变量和目标函数值
        """
        for vehicle_id in range(self.num_vehicles):
            self.calculate_battery_capacity_levels(vehicle_id)
            self.calculate_arrival_leave_wait_times(vehicle_id)
            self.calculate_travel_delay_wait_times(vehicle_id)

    def calculate_battery_capacity_levels(self, vehicle_id):
        """
        计算指定车辆在路径上每个节点的电池电量和容量水平
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        battery_levels = self.route_battery_levels[vehicle_id]
        capacity_levels = self.route_capacity_levels[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        battery_levels[0] = self.battery_capacity
        capacity_levels[0] = 0

        for i in range(1, len(route)):
            prev_node = route[i - 1]
            curr_node = route[i]
            # update the battery
            time = time_matrix[prev_node][curr_node]
            battery_levels[i] = battery_levels[i - 1] - time * self.battery_consume_rate
            # update the capacity
            capacity_levels[i] = capacity_levels[i - 1] + self.instance.demands[curr_node]

    def calculate_arrival_leave_wait_times(self, vehicle_id):
        """
        计算指定车辆在路径上每个节点的到达时间、离开时间和等待时间
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        arrival_times = self.route_arrival_times[vehicle_id]
        leave_times = self.route_leave_times[vehicle_id]
        wait_times = self.route_wait_times[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        arrival_times[0] = 0
        wait_times[0] = 0
        leave_times[0] = self.instance.service_times[route[0]]

        for i in range(1, len(route)):
            prev_node = route[i - 1]
            curr_node = route[i]
            arrival_times[i] = leave_times[i - 1] + time_matrix[prev_node][curr_node]
            wait_times[i] = max(0, self.instance.time_windows[curr_node][0] - arrival_times[i])
            if arrival_times[i] > self.instance.time_windows[curr_node][0]:
                wait_times[i] = 0
            leave_times[i] = arrival_times[i] + wait_times[i] + self.instance.service_times[curr_node]

    def calculate_travel_delay_wait_times(self, vehicle_id):
        """
        计算指定车辆的总行驶时间、总延误时间和总等待时间
        :param vehicle_id: 车辆 ID
        """
        route = self.routes[vehicle_id]
        leave_times = self.route_leave_times[vehicle_id]
        wait_times = self.route_wait_times[vehicle_id]
        time_matrix = self.instance.calculate_time_matrix()

        travel_time = 0
        delay_time = 0
        wait_time = 0

        for i in range(len(route) - 1):
            curr_node = route[i]
            next_node = route[i + 1]
            travel_time += time_matrix[curr_node][next_node]
            
            if self.instance.time_windows[next_node][1] != float('inf'):
                delay_time += max(0, leave_times[i] - self.instance.time_windows[next_node][1])
            
            wait_time += wait_times[i]

        self.total_travel_times[vehicle_id] = travel_time
        self.total_delay_times[vehicle_id] = delay_time
        self.total_wait_times[vehicle_id] = wait_time

    def objective_function(self):
        """
        计算目标函数值
        :return: 目标函数值（总行驶时间 + 总延误时间）
        """
        return np.sum(self.total_travel_times) + np.sum(self.total_delay_times)

    def is_feasible(self):
        """
        检查解的可行性
        """
        selected_vehicles = self.get_selected_vehicles()
        if not selected_vehicles:
            return False
        if not self.check_capacity_constraint(selected_vehicles):
            return False
        if not self.check_battery_constraint(selected_vehicles):
            return False
        # if not self.check_unique_service(selected_vehicles):
        #     return False
        # if not self.check_depot_constraint(selected_vehicles):
        #     return False
        # if not self.check_pickup_delivery_order(selected_vehicles):
        #     return False
        return True

    def get_selected_vehicles(self):
        """
        获取被选择的车辆列表
        :return: 被选择的车辆 ID 列表
        """
        selected_vehicles = []
        for vehicle_id in range(self.num_vehicles):
            if self.routes[vehicle_id] is not None and len(self.routes[vehicle_id]) > 2:
                selected_vehicles.append(vehicle_id)
        return selected_vehicles

    def check_capacity_constraint(self, selected_vehicles):
        """
        检查容量约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            if np.any(self.route_capacity_levels[vehicle_id] > self.vehicle_capacity):
                return False
        return True

    def check_battery_constraint(self, selected_vehicles):
        """
        检查电池约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            if np.any(self.route_battery_levels[vehicle_id] < 0):
                return False
        return True

    def check_unique_service(self, selected_vehicles):
        """
        检查唯一服务约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        visited_nodes = set()
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            for node in route:
                if node in visited_nodes:
                    return False
                visited_nodes.add(node)
        return True

    def check_depot_constraint(self, selected_vehicles):
        """
        检查车辆是否从车场出发并返回车场
        :param selected_vehicles: 被选择的车辆 ID 列表
        """
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            if route[0] != 0 or route[-1] != 0:
                return False
        return True

    def check_pickup_delivery_order(self, selected_vehicles):
        """
        检查取货-送货顺序约束
        :param selected_vehicles: 被选择的车辆 ID 列表
        :return: True 如果取货-送货顺序约束满足，False 否则
        """
        for vehicle_id in selected_vehicles:
            route = self.routes[vehicle_id]
            visited_pickups = set()
            for node in route:
                if 1 <= node <= self.instance.n:
                    visited_pickups.add(node)
                elif self.instance.n < node <= 2 * self.instance.n:
                    if node - self.instance.n not in visited_pickups:
                        return False
        return True

    def plot_solution(self, vehicle_ids=None):
        """
        绘制解的路线图
        :param vehicle_ids: 要绘制路径的车辆 ID 列表（可选），默认为 None，表示绘制所有被选择车辆的路径
        """
        selected_vehicles = self.get_selected_vehicles()
        if not selected_vehicles:
            print("No vehicle is selected in the solution.")
            return
    
        if vehicle_ids is None:
            vehicle_ids = selected_vehicles
        else:
            vehicle_ids = [vehicle_id for vehicle_id in vehicle_ids if vehicle_id in selected_vehicles]
    
        if not vehicle_ids:
            print("No valid vehicle IDs provided.")
            return
    
        plt.figure(figsize=(8, 8))
        plt.scatter(self.instance.depot[0], self.instance.depot[1], c='red', marker='s', s=100, label='Depot')
        plt.scatter([p[0] for p in self.instance.pickup_points], [p[1] for p in self.instance.pickup_points], c='blue', marker='o', label='Pickup')
        plt.scatter([d[0] for d in self.instance.delivery_points], [d[1] for d in self.instance.delivery_points], c='green', marker='d', label='Delivery')
    
        plt.annotate('Depot', (self.instance.depot[0], self.instance.depot[1]), textcoords='offset points', xytext=(0, 10), ha='center')
        for i, pickup_point in enumerate(self.instance.pickup_points, start=1):
            plt.annotate(f'P{i}', (pickup_point[0], pickup_point[1]), textcoords='offset points', xytext=(0, 5), ha='center')
        for i, delivery_point in enumerate(self.instance.delivery_points, start=1):
            plt.annotate(f'D{i}', (delivery_point[0], delivery_point[1]), textcoords='offset points', xytext=(0, 5), ha='center')
    
        color_map = plt.cm.get_cmap('viridis', len(vehicle_ids))
        for i, vehicle_id in enumerate(vehicle_ids):
            route = self.routes[vehicle_id]
            color = color_map(i)
            
            route_points = [self.instance.depot] + [self.instance.pickup_points[node-1] if node <= self.instance.n else self.instance.delivery_points[node-self.instance.n-1] for node in route[1:-1]] + [self.instance.depot]
            route_x = [point[0] for point in route_points]
            route_y = [point[1] for point in route_points]
            
            plt.plot(route_x, route_y, color=color, linestyle='-', linewidth=1.5, alpha=0.8, label=f'Vehicle {vehicle_id+1}')
    
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('PDPTW Solution')
        plt.legend()
        plt.grid(True)
        plt.show()

In [4]:
import pandas as pd
import numpy as np

def insertion_heuristics(df, num_vehicles=None, vehicle_capacity=None, total_orders=None, penalty=None, speed=None, battery=None):
    
    orders = df
    total_orders = int(orders.shape[0]/2-1)

    if num_vehicles == None:
        num_vehicles = 5
    else:
        num_vehicles = num_vehicles
    
    if vehicle_capacity == None:
        vehicle_capacity = 6
    else:
        vehicle_capacity = vehicle_capacity

    if penalty == None:
        penalty = 2
    else:
        penalty = penalty
    
    if speed == None:
        speed = 4
    else:
        speed = speed

    if battery == None:
        battery = 20
    else:
        battery = battery
    # service = 2

    # Extract necessary information from the DataFrame
    pickup_delivery_pairs = orders[['ID', 'X', 'Y']].values
    earliest = orders[['ReadyTime']].values
    latest = orders[['DueDate']].values
    demand = orders[['Demand']].values
    x = orders[['X']].values
    y = orders[['Y']].values
    services = orders[['ServiceTime']].values

    # Initialize the routes, start and end with depot
    routes = [[0, 0] for _ in range(num_vehicles)]

    # Initialize the capacities and time for each vehicle
    capacity_dict = {0: 0}
    time_dict = {0: 0}

# Function to calculate Euclidean distance
    def euclidean_distance(p1, p2):
        return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Insertion heuristic algorithm
    def insert_request(routes, pickup_delivery_pairs, earliest, latest, demand, capacity_dict, time_dict):
        for request in pickup_delivery_pairs[1:(total_orders+1)]:  #Enumerate number of orders
            best_vehicle = None
            best_pickup_position = None
            best_dropoff_position = None
            best_cost = float('inf')

            for vehicle_idx, route in enumerate(routes):
                for i in range(1, len(route)):
                    new_route = route[:i] + [int(request[0])] + route[i:]
                    for j in range(i+1, len(new_route)):
                        new_route = new_route[:j] + [int(request[0]+total_orders)] + new_route[j:]
                        new_time_dict = time_dict.copy()
                        new_capacity_dict = capacity_dict.copy()
                        new_time_dict, new_capacity_dict = update_node(new_route, earliest, new_time_dict, new_capacity_dict)
                        dist = total_distance(new_route)
                        if any(value > vehicle_capacity for value in new_capacity_dict.values()) or (dist > battery):
                            continue
                        else:
                            new_routes = routes.copy()
                            new_routes[vehicle_idx] = new_route
                            cost = calculate_route_cost(new_routes, new_time_dict)
                            if cost < best_cost:
                                best_vehicle = vehicle_idx
                                best_pickup_position = i
                                best_dropoff_position = j
                                best_cost = cost

            if best_vehicle is not None:
                routes[best_vehicle].insert(best_pickup_position, int(request[0]))
                routes[best_vehicle].insert(best_dropoff_position, int(request[0]+total_orders))

                time_dict, capacity_dict = update_node(routes[best_vehicle], earliest, time_dict, capacity_dict)
            else:
                print('Need more vehicles!')

    def calculate_route_cost(routes, time_dict):
        cost = 0
        for vehicle_idx, route in enumerate(routes):
            for i in range(len(route) - 1):
                cost += euclidean_distance([x[route[i]], y[route[i]]],[x[route[i+1]], y[route[i+1]]])
            for i in time_dict:
                if time_dict[i] > latest[i]:
                    cost += penalty  #penalty cost
        return cost

    def update_node(route, earliest, time_dict, capacity_dict):
        for idx, node in enumerate(route[:-1]):
            if idx == 0:
                continue
            time_dict[node] = max(earliest[node],
                                    time_dict[route[idx-1]] + services[route[idx-1]] + euclidean_distance([x[route[idx-1]], y[route[idx-1]]],[x[route[idx]], y[route[idx]]])/speed*60)[0] #speed is 4
            capacity_dict[node] = (capacity_dict[route[idx-1]] + demand[node])[0]
        return time_dict, capacity_dict

    def total_distance(route):
        dist = 0
        for i in range(len(route) - 1):
                dist += euclidean_distance([x[route[i]], y[route[i]]],[x[route[i+1]], y[route[i+1]]])
        return dist

# Execute the insertion heuristic
    insert_request(routes, pickup_delivery_pairs, earliest, latest, demand, capacity_dict, time_dict)

    print(routes)

    return routes

In [6]:
import random
import numpy as np
import matplotlib.pyplot as plt

# 创建一个 PDPTWInstance 对象
instance = PDPTWInstance(n=20, map_size=3, speed=4, extra_time=10,seed = 1234)
df = instance.to_dataframe()
d_last = []
d_last.append([int(max(df.ID)+1),'d',df.X[0],df.Y[0],0,0,float('inf'),0,0])
columns = ['ID', 'Type', 'X', 'Y', 'Demand', 'ReadyTime', 'DueDate', 'ServiceTime', 'PartnerID']
d_last = pd.DataFrame(d_last, columns=columns)
df = pd.concat([df, d_last], axis = 0)

routes = insertion_heuristics(df, battery = 25)

[[0, 1, 11, 31, 21, 15, 35, 12, 32, 9, 18, 38, 29, 0], [0, 4, 24, 2, 17, 37, 22, 20, 40, 0], [0, 8, 28, 3, 13, 33, 23, 0], [0, 5, 25, 10, 30, 14, 16, 36, 34, 19, 39, 0], [0, 6, 7, 27, 26, 0]]


### Create order distance table

In [10]:
df_orders = df.iloc[1:-1,:]
df_orders.head()

,ID,Type,X,Y,Demand,ReadyTime,DueDate,ServiceTime,PartnerID
1,1,cp,2.798721,-0.355604,1,62,inf,10,21
2,2,cp,2.635614,0.493365,1,104,inf,10,22
3,3,cp,1.598886,-1.579141,1,32,inf,7,23
4,4,cp,-0.923466,0.739689,1,24,inf,10,24
5,5,cp,-1.901456,-2.313522,1,13,inf,6,25


In [29]:
n_orders = int(df_orders.shape[0]/2)
stops = df_orders[['X','Y']].values
times = df_orders[['ReadyTime']].values
speed = 4

def euclidean_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

d_matrix = []
for i in range(n_orders):
    d_list = []
    for j in range(n_orders):
        d_1 = euclidean_distance(stops[i],stops[j]) # pickup-pickup
        d_2 = euclidean_distance(stops[i],stops[j+n_orders]) #pickup-dropoff
        d_3 = euclidean_distance(stops[i+n_orders],stops[j]) #dropoff-pickup
        d_4 = euclidean_distance(stops[i+n_orders],stops[j+n_orders]) #dropoff-dropoff
        d_avg = (d_1 + d_2 + d_3 + d_4)/4

        t = abs(times[i] - times[j])/60*speed
        d_total = d_avg + t[0]
        d_list.append(d_total)
    d_matrix.append(d_list)


### SISR process

In [44]:
# Generate number of strings to remove
def number_of_strings(L_max, avg_remove_order, routes, n_orders):
    T_avg = np.floor(n_orders/len(routes))
    l_s_max = min(T_avg, L_max)
    k_s_max = 4*avg_remove_order/(1+l_s_max) - 1
    k_s = np.floor(np.random.uniform(1, k_s_max + 1))
    
    return k_s, l_s_max

# Generate number of orders to remove in each string
def number_of_orders(l_s_max, route):
    l_t_max = min(l_s_max, (len(route)-2)/2)
    l_t = np.floor(np.random.uniform(1, l_t_max + 1))
    
    return l_t

# Generate random order to start, remove orders according to order distance matrix
def order_removal(d_matrix, ):